Nội dung Lý Thuyết:

- Hệ Phân Tán (Distributed System)
- Concurency vs Paralism
- MapReduce Framework
- Spark Framework Features: `In-memory`, `Lazy`, `Fault-Tolerance`
- RDD (Resilient Distributed Dataset)
- DataFrame, Compare with RDD

**CHÚ Ý:**
Các Bài tập dưới đây sẽ sử dụng file `1M.json` để làm. Nếu bạn chưa upload file `1M.json` vào folder `dbfs:/Filestore/wavepoint`, hãy đọc file hướng dẫn upload file tại Classroom và chú ý kiểm tra xem đã bật `DBFS File Browser` trong `Settings` hay chưa

thực hiện load file sử dụng spark tại `dbfs:/Filestore/wavepoint/1M.json`

In [0]:
val path = "dbfs:/FileStore/wavepoint/1M.json"

val input = spark
  .read
  .json(path)

path: String = dbfs:/FileStore/wavepoint/1M.json
input: org.apache.spark.sql.DataFrame = [MaxValidSpeed: double, ProviderId: bigint ... 13 more fields]

sử dụng hàm `display()` để hiển thị kết quả, **Lưu ý**: hàm `display()` chỉ chạy được trên Databricks và hỗ trợ hiển thị kết quả và visualize dữ liệu 

In [0]:
display(input.limit(10))

MaxValidSpeed,ProviderId,TimeToBeginAck,TimeToEndAck,Vss,_id,datetime,driver,heading,isOverSpeed,isOverSpeed2,speed,vehicle,x,y
120.0,1028,List(1563753600),609.376,0.0,List(5d349a13c90a9713bcde22ea),1563753660,920139002101,88.0,false,false,0.0,51D50375,106.6105233333,10.8404816667
120.0,1028,List(1563753600),625.0032,0.0,List(5d349a13c90a9713bcde22f5),1563753660,,0.0,false,false,0.0,77C09121,109.201845,13.78429
120.0,1028,List(1563753600),625.0032,0.0,List(5d349a13c90a9713bcde22f6),1563753660,,160.0,false,false,0.0,77A13543,106.800465,10.935161666700001
120.0,1028,List(1563753600),640.6276,19.0,List(5d349a13c90a9713bcde2301),1563753661,,308.0,false,false,24.0,30E49697,105.8307766667,21.01754
120.0,1028,List(1563753600),640.6276,0.0,List(5d349a13c90a9713bcde2304),1563753660,,332.0,false,false,0.0,77C09956,109.0845383333,13.919595
120.0,1028,List(1563753600),640.6276,0.0,List(5d349a13c90a9713bcde2308),1563753660,790121793706,66.0,false,false,0.0,44LD00456,108.1957833333,16.0067733333
90.0,1028,List(1563753600),640.6276,0.0,List(5d349a13c90a9713bcde230a),1563753660,,218.0,false,false,0.0,77C08269,108.9222616667,13.90731
120.0,1028,List(1563753600),656.251,0.0,List(5d349a13c90a9713bcde230c),1563753660,,138.0,false,false,0.0,51D12799,106.31844,10.4279316667
90.0,1028,List(1563753600),671.8815000000001,23.0,List(5d349a13c90a9713bcde230e),1563753661,,308.0,false,false,28.0,30E09688,105.368605,20.239485
120.0,1028,List(1563753600),687.5012,0.0,List(5d349a13c90a9713bcde2313),1563753660,,270.0,false,false,0.0,51C85009,109.2007683333,12.2807983333


Câu 1: Có bao nhiêu bản ghi có `speed > 0` ?

In [0]:
import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions._

In [0]:
val lesson1 = input.filter($"speed" > 0).count

lesson1

lesson1: Long = 136128
res5: Long = 136128

Câu 2: Có bao nhiêu ProviderId duy nhất trong `input`

In [0]:
val lesson2 = input.select($"ProviderId").distinct().count()

lesson2

lesson2: Long = 58
res7: Long = 58

Câu 3: Hãy lọc biển số xe của tỉnh và đếm số lượng xe của mỗi tỉnh (biển số xe tỉnh là 2 số đầu của trường `vehicle` VD: Với xe có biển số: `29A12345` thì số `29` là biển số của tỉnh)

In [0]:
val lesson3 = input.withColumn(
  "vehicle_prefix",
  substring(col("vehicle"), 1, 2)
)
val result = lesson3.groupBy($"vehicle_prefix").agg(count("*") as "count")
display(result)

vehicle_prefix,count
51,226165
54,10343
15,10133
11,749
29,97591
69,3661
73,4541
64,6903
30,48165
34,19922


Khuyến khích làm thêm

Câu 4: Hãy lấy giá trị kinh độ `lon` (`x`) và vĩ độ `lat`(`y`) và sử dụng để gọi tới nominatimAPI để lấy giá trị `display_name` từ API và hiển thị ra kết quả. Đầu ra mong muốn (`x`, `y`, `display_name`). **Chú ý lấy limit 100 bản ghi**

In [0]:
%sh
curl --location 'https://nominatim.openstreetmap.org/reverse?format=json&lat=10.8309183333&lon=106.57334'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   811  100   811    0     0   3975      0 --:--:-- --:--:-- --:--:--  3975
{"place_id":238777979,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright","osm_type":"way","osm_id":32576575,"lat":"10.83123646694974","lon":"106.5733607607416","class":"highway","type":"tertiary","place_rank":26,"importance":0.05340036697947658,"addresstype":"road","name":"Thới Hòa","display_name":"Thới Hòa, Khu công nghiệp Vĩnh Lộc, Xã Vĩnh Lộc A, Huyện Bình Chánh, Thành phố Hồ Chí Minh, 71821, Việt Nam","address":{"road":"Thới Hòa","industrial":"Khu công nghiệp Vĩnh Lộc","village":"Xã Vĩnh Lộc A","city_district":"Huyện Bình Chánh","city":"Thành phố Hồ Chí Minh","ISO3166-2-lvl4":"VN-SG","postcode":"71821","country":"Việt Nam","country_code":"v

In [0]:
val lesson4 =

display(lesson4)